In [7]:
import pandas as pd
import time
import datetime
import redis
from flask import current_app
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [8]:
#!/usr/bin/python
import MySQLdb

db = MySQLdb.connect(host="localhost",    # your host, usually localhost
                     user="root",         # your username
                     #passwd="xpx",  # your password
                     db="xudatabase")        # name of the data base

cur = db.cursor()

In [9]:
# import article data from mysql db 
def import_data_articles(table_name):
    #ds = pd.read_csv(data_source)
    ds = cur.execute('select * from ' +table_name)
    r = cur.fetchall()
    return pd.DataFrame(list(r),columns=['PMID', 'abstract'])
    

In [10]:
userId =1
userpool = pd.read_csv("./data/testdata.csv")
ds = cur.execute('select * from ' +table_name)
    r = cur.fetchall()
    return pd.DataFrame(list(r),columns=['PMID', 'abstract'])



In [11]:
dailypool = import_data_articles('articles')

In [15]:
def train_time():
    start = time.time()
    train(userpool, dailypool)
    print("Engine trained in %s seconds." % (time.time() - start))    

In [16]:
# refactor the training logic into this function
def train(userpool, dailypool):
    ds = pd.concat([userpool, dailypool])
    ds.reset_index(drop=True, inplace=True)
    
    #clean up not ASCII chars
    ds['abstract'] = ds["abstract"].apply(lambda x: ''.join([" " if ord(i) < 32 or ord(i) > 126 else i for i in x]))
    
    tf = TfidfVectorizer(analyzer='word',
                             ngram_range=(1, 3),
                             min_df=0,
                             stop_words='english')
    tfidf_matrix = tf.fit_transform(ds['abstract'])
    cosine_similarities = linear_kernel(tfidf_matrix[:userpool.shape[0]], tfidf_matrix[:])
    similarities = pd.DataFrame(cosine_similarities)
    
    prediction_matrix = pd.DataFrame(data=userpool.PMID, columns=['PMID'], index=userpool.index)
    similarity_scores = prediction_matrix.copy()
    
    top_n = 12
    if (ds.shape[0] < top_n):
        top_n = ds.shape[0] + 1

    for i in range(1,top_n-1):
        col_name = i
        prediction_matrix[col_name] = None
        similarity_scores[col_name] = -1

    now = datetime.datetime.now()
    
    for idx in range(userpool.shape[0]):
        #print "idx: %s" % idx
        similar_indices = cosine_similarities[idx].argsort()[:-top_n:-1]
        #print "similar indices: %s" % similar_indices
        similar_items = [(cosine_similarities[idx][i], ds['PMID'][i]) for i in similar_indices]
        #print "similar items: \n %s" % similar_items
        for i in range(1,top_n-1): 

            prediction_matrix.ix[idx, i] = similar_items[i][1] # insert into db directly 
            similarity_scores.ix[idx, i] = similar_items[i][0] # insert into a db table with date

        query = 'insert into recommend_articles values ("%s", %d, %d,\
            %d, %d, %d, %d, %d,\
            %d, %d, %d, %d, %d,\
            %.6f,%.6f,%.6f,%.6f,%.6f,\
            %.6f,%.6f,%.6f,%.6f,%.6f)'%(now, userId,userpool.ix[idx,'PMID'],\
                                                         similar_items[1][1],similar_items[2][1],\
                                                         similar_items[3][1],similar_items[4][1],similar_items[5][1],\
                                                         similar_items[6][1],similar_items[7][1],similar_items[8][1],\
                                                         similar_items[9][1],similar_items[10][1],similar_items[1][0],\
                                                         similar_items[2][0],\
                                                         similar_items[3][0],similar_items[4][0],similar_items[5][0],\
                                                         similar_items[6][0],similar_items[7][0],similar_items[8][0],\
                                                         similar_items[9][0],similar_items[10][0])

        cur.execute(query)
        db.commit()
    return 1;

In [17]:
calculate_time()

idx: 0
similar indices: [  0   1  54 114 716 722 477 430 623 583 644]
similar items: 
 [(0.99999999999999933, 27270041), (0.083793924816965695, 19465921), (0.081937461746571905, 28274158), (0.072891501922273075, 28274224), (0.044891564175493434, 28274951), (0.041857678838202947, 28274958), (0.041100569768590646, 28274652), (0.036343454977238954, 28274595), (0.034456518980895613, 28274838), (0.033416943214084893, 28274794), (0.033103645640632812, 28274862)]
idx: 1
similar indices: [  1   0 446  53 283 716  89 565  54 159 639]
similar items: 
 [(1.0000000000000002, 19465921), (0.083793924816965695, 27270041), (0.044319576133340331, 28274615), (0.036427143370675, 28274157), (0.025591237986448155, 28274419), (0.024294734106412176, 28274951), (0.023544763988480065, 28274198), (0.02349890390335941, 28274768), (0.02340229811863526, 28274158), (0.022885901792000285, 28274274), (0.022311327257618148, 28274857)]
idx: 2
similar indices: [  2   3 808 116 468 807 777 622 646 326 664]
similar items:

In [ ]:
db.close()

In [214]:
cur.execute('insert into last_pmid values ("%s", 27270041)'%now)
db.commit()
    